### Employee Map Automation Tool

##### Updates Note

--------------------------------------------------------------------------------------------

#### Ver.4  |  July 14th, 2022

* Arcpy
    * Median Center (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/spatial-statistics/median-center.htm)
    * Mean Center (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/spatial-statistics/mean-center.htm)
    * Select Layer By Location (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/select-layer-by-location.htm)
    * ArcGISProject (arcpy.mp) (https://pro.arcgis.com/en/pro-app/2.8/arcpy/mapping/arcgisproject-class.htm)
    * addDataFromPath (data_path) (https://pro.arcgis.com/en/pro-app/2.8/arcpy/mapping/map-class.htm)
    * Symbology (https://pro.arcgis.com/en/pro-app/2.8/arcpy/mapping/symbology-class.htm)
    * env (https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-classes/env.htm)
    * DataFrame (https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-mapping/dataframe-class.htm)
<br>
* Other references
    * Add feautre class to Map in Current project (https://gis.stackexchange.com/questions/265592/adding-feature-class-to-map-in-current-project-using-arcpy-in-arcgis-pro)
    * Add feature class to Map in ArcGIS Pro (https://community.esri.com/t5/python-questions/add-feature-class-to-map-in-arcgis-pro/td-p/64852)
    * Add data into ArcMap from geodatabase using Python (https://community.esri.com/t5/data-management-questions/add-data-into-arcmap-from-geodatabase-using-python/td-p/18311)
    * About dataframes and layers (https://desktop.arcgis.com/en/arcmap/latest/extensions/arcreader/about-data-frames-and-layers.htm)
    * Python: Introduction to Map AUtomation (https://www.esri.com/content/dam/esrisites/en-us/about/events/media/UC-2019/technical-workshops/tw-6090-816.pdf)


<br>
1. Creating Mean Center except outliers <br>
    1) Create a "Meidan Center Point" with pop-centered employee points <br>
    2) Select the points within the search radius to exclude outliers outside of the range <br>
    3) Create a "Mean Center Point" with the selected employee poitns within the search radius <br>
<br>
2. Rendering a number of employee by zipcodes layer <br>
    1) Create variables for the current project, map, and the number of employee by zipcodes layer <br> 
    2) Update the layer rendering from simple to graduated colors <br>
    3) Set the break count as 5 and colorRamp as "Yellow-Orange-Red (Continuous)" <br>

--------------------------------------------------------------------------------------------

#### Ver.5

<br>
1. Rendering the Mean Center Point <br>
2. Create a finished layout using Layout, Bookmark, and Legend <br>
3. Export PDF <br>

##### July 14th, 2022 - EMA Ver.4

In [ ]:
# Employee Map Automization Tool 
# 220714 - Ver.4

import arcpy
from arcpy import env
import os
import sys

# set environment setting
arcpy.env.overwriteOutput = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
src_dist_lun = arcpy.GetParameterAsText(6) # linear unit
emp_count_xlsx = arcpy.GetParameterAsText(7) # xlsx

# Copy rows to create OID in memory
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
emp_median_center = os.path.join(workspace, "emp_median_center")
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_median_center,
)

# Remove outliers using the distance
emp_selected = os.path.join(workspace, "emp_selected")
emp_selected = arcpy.SelectLayerByLocation_management(
    in_layer = xy_point_fly,
    overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
    select_features = emp_median_center,
    search_distance = src_dist_lun,
    selection_type = "NEW_SELECTION"
)

# Create a Mean Center Point with the Selected Employee Points
arcpy.MeanCenter_stats(
    Input_Feature_Class = emp_selected,
    Output_Feature_Class = emp_mean_fly
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Add the Zipcode Feature layer to Map
aprx = arcpy.mp.ArcGISProject("CURRENT") # Current Project
aprxMap = aprx.listMaps()[0] # Default Map
emp_zip_Lyr = os.path.join(workspace, "Emp_Zipcode")
aprxMap.addDataFromPath(emp_zip_Lyr)
aprxLyr = aprxMap.listLayers()[0]

# Rendering the Zipcode Feature layer
sym = aprxLyr.symbology
if hasattr(sym, 'renderer'):
    if sym.renderer.type == 'SimpleRenderer':
        sym.updateRenderer('GraduatedColorsRenderer')
        sym.renderer.classificationField = 'Join_Count'
        sym.renderer.breakCount = 5
        sym.renderer.colorRamp = aprx.listColorRamps('Yellow-Orange-Red (Continuous)')[0]
        
        aprxLyr.symbology = sym

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)
arcpy.management.Delete(emp_median_center)
arcpy.management.Delete(emp_selected)

In [ ]:
# Employee Map Automization Tool 
# 220714 - Ver.4

import arcpy
from arcpy import env
import os
import sys

# set environment setting
arcpy.env.overwriteOutput = True
#arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
src_dist_lun = arcpy.GetParameterAsText(6) # linear unit
emp_count_xlsx = arcpy.GetParameterAsText(7) # xlsx

# Copy rows to create OID in memory
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
emp_median_center = os.path.join(workspace, "emp_median_center")
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_median_center,
)

# Remove outliers using the distance
emp_selected = os.path.join(workspace, "emp_selected")
emp_selected = arcpy.SelectLayerByLocation_management(
    in_layer = xy_point_fly,
    overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
    select_features = emp_median_center,
    search_distance = src_dist_lun,
    selection_type = "NEW_SELECTION"
)

# Create a Mean Center Point with the Selected Employee Points
arcpy.MeanCenter_stats(
    Input_Feature_Class = emp_selected,
    Output_Feature_Class = emp_mean_fly
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Add the Zipcode Feature layer to Map
aprx = arcpy.mp.ArcGISProject("CURRENT") # Current Project
aprxMap = aprx.listMaps()[0] # Default Map
emp_zip_Lyr = os.path.join(work_dbs, "Emp_Zipcode")
aprxMap.addDataFromPath(emp_zip_Lyr)
aprxLyr = aprxMap.listLayers()[0]

# Rendering the Zipcode Feature layer
sym = aprxLyr.symbology
if hasattr(sym, 'renderer'):
    if sym.renderer.type == 'SimpleRenderer':
        sym.updateRenderer('GraduatedColorsRenderer')
        sym.renderer.classificationField = 'Join_Count'
        sym.renderer.breakCount = 5
        sym.renderer.colorRamp = aprx.listColorRamps('Yellow-Orange-Red (Continuous)')[0]
        
        aprxLyr.symbology = sym

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)
arcpy.management.Delete(emp_median_center)
arcpy.management.Delete(emp_selected)

In [19]:
# 220713
# Rendering the ZIP_Code layer to Graduated Colors with short version

import arcpy, os, sys
arcpy.env.workspace = "C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb"
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxMap = aprx.listMaps()[0]

emp_zip_fly = "C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\Emp_Zipcode"
aprxMap.addDataFromPath(emp_zip_fly)
aprxLyr = aprxMap.listLayers()[0]
lyt = aprx.listLayouts()[0]
mf = lyt.listElements()[0]

# Rendering Symbology
sym = aprxLyr.symbology

#if hasattr(sym, 'renderer'):
#    if sym.renderer.type == 'SimpleRenderer':
#        sym.updateRenderer('GraduatedColorsRenderer')
#        sym.renderer.classificationField = 'Join_Count'
#        sym.renderer.breakCount = 5
#        sym.renderer.colorRamp = aprx.listColorRamps('Yellow-Orange-Red (Continuous)')[0]
        
#        aprxLyr.symbology = sym

#bkmk = mf._mp.listBookmarks()[0]
#mf.zoomToBookmark(bkmk)
#lyt.exportToPDF(r"C:\ArcGIS\Projects\Dev_EMA\Test12.pdf")


In [3]:
import os
work_dbs = r'C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb'
print(os.path.join(work_dbs, "Emp_Zipcode"))

C:\ArcGIS\Projects\Dev_EMA\Dev_EMA.gdb\Emp_Zipcode


In [ ]:
# EMA Ver.4
# Updates Note
# 220713 
# 1. Creating Mean Center
#   1) Create meidan center
#   2) Capture the poitns within the radius to exclude outliers
#   3) Create mean center point with the selected pop-centered employee points
# 2. Rendering the employee count layer 


import arcpy
from arcpy import env
import os

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
src_dist_lun = arcpy.GetParameterAsText(6) # linear unit
emp_count_xlsx = arcpy.GetParameterAsText(7) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
emp_median_center = os.path.join(workspace, "emp_median_center")
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_median_center,
)

# Remove outliers using the distance
emp_selected = os.path.join(workspace, "emp_selected")
emp_selected = arcpy.SelectLayerByLocation_management(
    in_layer = xy_point_fly,
    overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
    select_features = emp_median_center,
    search_distance = src_dist_lun,
    selection_type = "NEW_SELECTION"
)

# Create a Mean Center Point with the Selected Employee Points
arcpy.MeanCenter_stats(
    Input_Feature_Class = emp_selected,
    Output_Feature_Class = emp_mean_fly
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)
arcpy.management.Delete(emp_median_center)
arcpy.management.Delete(emp_selected)

##### July 13th, 2022 - EMA Ver.4

In [ ]:
# EMA Ver.4
# Updates Note
# 220713 
# 1. Creating Mean Center
#   1) Create meidan center
#   2) Capture the poitns within the radius to exclude outliers
#   3) Create mean center point with the selected pop-centered employee points
# 2. Rendering the employee count layer 


import arcpy
from arcpy import env
import os

# set environment setting
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
emp_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
emp_field = arcpy.GetParameterAsText(2) # employee csv field
xy_point_fly = arcpy.GetParameterAsText(3) # feature layer
emp_zip_fly = arcpy.GetParameterAsText(4) # feature layer
emp_mean_fly = arcpy.GetParameterAsText(5) # feature layer
src_dist_lun = arcpy.GetParameterAsText(6) # linear unit
emp_count_xlsx = arcpy.GetParameterAsText(7) # xlsx

# Copy rows to create OID in memory
# set local variable
arcpy.env.workspace = work_dbs
workspace = work_dbs
emp_table = os.path.join(workspace, "emp_table")
pop_table = os.path.join(workspace, "pop_table") 
pop_csv = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/PopCenterZips_March2022_csv.csv"
arcpy.CopyRows_management(emp_csv, emp_table)
arcpy.CopyRows_management(pop_csv, pop_table)

# Join the two table
# Set local variable
arcpy.JoinField_management(emp_table, emp_field, 
                           pop_table, "ZipCode")

# Copy the outcome
join_table = os.path.join(workspace, "join_table")
arcpy.Copy_management(emp_table, join_table)

# Create point features
xy_point_shp = "point.shp"
arcpy.management.XYTableToPoint(
    in_table = join_table, 
    out_feature_class = xy_point_shp,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# Copy feature layer from shp
arcpy.CopyFeatures_management(xy_point_shp, xy_point_fly)

# Spatial join with ZipCode Files
zip_polyg_fly = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/ZIPCodes_zp"
arcpy.SpatialJoin_analysis(
    target_features = zip_polyg_fly, 
    join_features = xy_point_fly, 
    out_feature_class = emp_zip_fly, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Spatial join with NY Area Count
emp_count = os.path.join(workspace, "emp_count")
ny_region_count = "N:/Team-GISData/Projects/NewYork/2022/2stQtr/Temp/EDA_tool.gdb/NY_Area_Region_Count"
arcpy.SpatialJoin_analysis(
    target_features = ny_region_count, 
    join_features = xy_point_fly, 
    out_feature_class = emp_count, 
    join_operation = "JOIN_ONE_TO_ONE", 
    join_type = 0)

# Employee Meidan Center
emp_median_center = os.path.join(workspace, "emp_median_center")
arcpy.MedianCenter_stats(
    Input_Feature_Class = xy_point_fly,
    Output_Feature_Class = emp_median_center,
)

# Remove outliers using the distance
emp_selected = os.path.join(workspace, "emp_selected")
emp_selected = arcpy.SelectLayerByLocation_management(
    in_layer = xy_point_fly,
    overlap_type = "WITHIN_A_DISTANCE_GEODESIC",
    select_features = emp_median_center,
    search_distance = src_dist_lun,
    selection_type = "NEW_SELECTION"
)

# Create a Mean Center Point with the Selected Employee Points
arcpy.MeanCenter_stats(
    Input_Feature_Class = emp_selected,
    Output_Feature_Class = emp_mean_fly
)

# Table to excel
arcpy.TableToExcel_conversion(emp_count, emp_count_xlsx)

# Delete unnecessary data
arcpy.management.Delete(emp_table)
arcpy.management.Delete(pop_table)
arcpy.management.Delete(join_table)
arcpy.management.Delete("point.shp")
arcpy.management.Delete(emp_count)
arcpy.management.Delete(emp_median_center)
arcpy.management.Delete(emp_selected)